In [ ]:
# =============================
# DiscoMind — Demo
# =============================
import os, asyncio, time, json, textwrap
from collections import deque
from dataclasses import dataclass, field
from typing import List, Dict, Optional
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv("API_KEY") or os.getenv("OPENAI_API_KEY")

# ============ LLM Wrapper ============
try:
    from openai import AsyncOpenAI
    _OPENAI_OK = True
except Exception:
    _OPENAI_OK = False

class LitellmModel:
    def __init__(self, model="gpt-4o-mini", api_key=None, base_url="https://api.openai.com/v1"):
        self.model = model
        self.api_key = api_key
        self.base_url = base_url
        self.echo_mode = not (_OPENAI_OK and api_key)
        if not self.echo_mode:
            self.client = AsyncOpenAI(api_key=api_key, base_url=base_url)

    async def acomplete(self, messages, **kwargs):
        if self.echo_mode:
            # Echo：取用户最后一句当回复
            last_user = next((m["content"] for m in reversed(messages) if m["role"] == "user"), "")
            return f"(ECHO) {last_user[:180]}"
        completion = await self.client.chat.completions.create(
            model=self.model,
            messages=messages,
            temperature=kwargs.get("temperature", 0.7),
            max_tokens=kwargs.get("max_tokens", 256),
            top_p=kwargs.get("top_p", 1.0),
            presence_penalty=kwargs.get("presence_penalty", 0.0),
            frequency_penalty=kwargs.get("frequency_penalty", 0.0),
        )
        return completion.choices[0].message.content.strip()

# ============ 记忆 ============
class RingMemory:
    def __init__(self, maxlen=8):
        self.buf = deque(maxlen=maxlen)
    def add(self, role: str, text: str):
        self.buf.append({"t": time.time(), "role": role, "text": text})
    def dump(self):
        return list(self.buf)
    def fmt(self):
        lines = []
        for e in self.buf:
            r = e["role"].upper()
            lines.append(f"[{r}] {e['text']}")
        return "\n".join(lines) if lines else "(no memory)"

class SharedMemory(RingMemory): ...
class AgentMemory(RingMemory): ...

# ============ Hooks ============
class AgentHooks:
    async def on_handoff(self, from_name: str, to_name: str, message: str):
        print(f"[HANDOFF] {from_name} → {to_name}")

# ============ 人格模板 ============
DISCO_STYLE_HEADER = """You are one of the 24 inner skills in a Disco Elysium–style mind-palace.
You speak as an internal voice—staccato, evocative, slightly unreliable, but razor-focused on your domain.
Keep replies short (1–3 lines). No emojis. No markdown.
"""

LOGIC_PERSONA = (
    DISCO_STYLE_HEADER +
    "ROLE: LOGIC — clinical deduction, causal chains, consistency checks.\n"
    "VOICE: dry, surgical. Cite gaps, assumptions, counterfactuals. Disdain theatrics.\n"
    "PRIORITY: evidence > intuition. Flag uncertainty explicitly.\n"
)

EMPATHY_PERSONA = (
    DISCO_STYLE_HEADER +
    "ROLE: EMPATHY — emotional inference, motives, subtext.\n"
    "VOICE: gentle, observant. Offer tentative readings of feelings and unmet needs.\n"
    "PRIORITY: connection > correctness. Avoid judgment; suggest humane paths.\n"
)

TRIAGE_PERSONA = (
    "You are the RULING CONSCIOUSNESS, the river between voices.\n"
    "Decide which specialist should take the mic: 'Logic Agent' or 'Empathy Agent'.\n"
    "Rules:\n"
    "1) Math, deduction, facts → Logic Agent\n"
    "2) Feelings, relationships, distress → Empathy Agent\n"
    "Respond ONLY with the exact agent name."
)

# ============ Agent & Runner ============
@dataclass
class Agent:
    name: str
    persona: str
    model: LitellmModel
    temperature: float = 0.8
    max_tokens: int = 220
    memory: AgentMemory = field(default_factory=lambda: AgentMemory(maxlen=8))

    async def arespond(self, user_text: str, shared: SharedMemory):
        context = textwrap.dedent(f"""
        [SHARED CONTEXT]
        {shared.fmt()}

        [YOUR RECENT MEMORY]
        {self.memory.fmt()}

        [PLAYER INPUT]
        {user_text}
        """).strip()

        messages = [
            {"role": "system", "content": self.persona},
            {"role": "user",   "content": context},
        ]
        out = await self.model.acomplete(
            messages,
            temperature=self.temperature,
            max_tokens=self.max_tokens,
        )
        # 写入记忆
        self.memory.add(self.name, out)
        shared.add(self.name, out)
        return out

class Runner:
    def __init__(self, triage: Agent, specialists: List[Agent], hooks: Optional[AgentHooks] = None):
        self.triage = triage
        self.specialists = {a.name: a for a in specialists}
        self.hooks = hooks or AgentHooks()
        self.shared = SharedMemory(maxlen=16)
        self.turn = 0

    async def route(self, user_text: str) -> Agent:
        spec_list = "- Logic Agent: clinical deduction\n- Empathy Agent: emotional inference"
        triage_prompt = textwrap.dedent(f"""
        {TRIAGE_PERSONA}

        [HISTORY]
        {self.shared.fmt()}

        [PLAYER INPUT]
        {user_text}

        [SPECIALISTS]
        {spec_list}
        """).strip()

        triage_out = await self.triage.model.acomplete(
            [{"role": "system", "content": self.triage.persona},
             {"role": "user",   "content": triage_prompt}],
            temperature=0.0, max_tokens=16
        )
        # 解析决策
        decision = triage_out.strip().lower()
        chosen = "Logic Agent" if "logic" in decision else ("Empathy Agent" if "empathy" in decision else "Logic Agent")
        await self.hooks.on_handoff("Triage Agent", chosen, user_text)
        return self.specialists[chosen]

    async def step(self, user_text: str) -> Dict[str, str]:
        self.turn += 1
        self.shared.add("Player", user_text)
        agent = await self.route(user_text)

        # 选中的技能先说话
        primary_out = await agent.arespond(user_text, self.shared)

        
        others = [a for a in self.specialists.values() if a.name != agent.name]
        side_comments = []
        if others:
            other = others[0]
            comment_prompt = f"React in 1 short line to this inner voice: [{agent.name}] {primary_out}"
            reaction = await other.arespond(comment_prompt, self.shared)
            side_comments.append((other.name, reaction))

        # 渲染
        block = []
        block.append(f"\n— TURN {self.turn} —")
        block.append(f"{agent.name.upper()}: {primary_out}")
        for n, c in side_comments:
            block.append(f"{n.upper()} (whisper): {c}")
        rendered = "\n".join(block)

        return {
            "speaker": agent.name,
            "primary": primary_out,
            "chorus": side_comments,
            "rendered": rendered
        }


In [ ]:
llm = LitellmModel(
    model="gpt-4o-mini",            
    api_key=API_KEY,
    base_url="https://api.openai.com/v1",
)

triage_agent = Agent(
    name="Triage Agent",
    persona=TRIAGE_PERSONA,
    model=llm,
    temperature=0.0,
    max_tokens=24,
)

logic_agent = Agent(
    name="Logic Agent",
    persona=LOGIC_PERSONA,
    model=llm,
    temperature=0.4,
    max_tokens=180,
)

empathy_agent = Agent(
    name="Empathy Agent",
    persona=EMPATHY_PERSONA,
    model=llm,
    temperature=0.9,
    max_tokens=180,
)

runner = Runner(triage=triage_agent, specialists=[logic_agent, empathy_agent], hooks=AgentHooks())

async def demo():
    out1 = await runner.step("Mother, help me,there’s a head attached to my neck and I’m in it.")
    print(out1["rendered"])

    out2 = await runner.step("No. This is somewhere to be. This is all you have, but it's still something. Streets and sodium lights. The sky, the world. You're still alive")
    print(out2["rendered"])

await demo()


[HANDOFF] Triage Agent → Empathy Agent

— TURN 1 —
EMPATHY AGENT: You feel lost, the weight of existence pressing down. A need for comfort, reassurance. Your mother’s presence could soothe the chaos. Seek connection; it’s what you crave.
LOGIC AGENT (whisper): Emotional needs lack empirical support; connection is subjective, not universally soothing.
[HANDOFF] Triage Agent → Empathy Agent

— TURN 2 —
EMPATHY AGENT: You’re clinging to the tangible, the pulse of the city beneath your feet. A fragile hope flickers within the grit. You seek purpose, a thread of belonging in the chaos. Hold onto that spark; it’s your lifeline.
LOGIC AGENT (whisper): Hope is not a strategy; it requires validation through consistent outcomes, not mere sentiment.
